# Embedding Model Comparison: Word Disambiguation
Comparing how different models interpret the word 'fan' in different contexts

In [3]:
import boto3
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json
import pandas as pd

In [4]:
# Initialize Bedrock client
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

In [5]:
# Test contexts for "fan"
contexts = {
    "simple": "fan",
    "cooling_device": "The electric fan helps circulate air and cool the room during hot summer days.",
    "sports_enthusiast": "She is a huge fan of the basketball team and never misses a game.",
    "ceiling_fan": "Install a ceiling fan to improve airflow in the bedroom.",
    "music_fan": "He's been a fan of jazz music since he was a teenager."
}

print("Test contexts:")
for name, text in contexts.items():
    print(f"{name}: {text}")

Test contexts:
simple: fan
cooling_device: The electric fan helps circulate air and cool the room during hot summer days.
sports_enthusiast: She is a huge fan of the basketball team and never misses a game.
ceiling_fan: Install a ceiling fan to improve airflow in the bedroom.
music_fan: He's been a fan of jazz music since he was a teenager.


In [6]:
# Embedding models to compare
models = [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-text-v2:0", 
    "cohere.embed-english-v3",
    "cohere.embed-multilingual-v3"
]

print(f"Testing {len(models)} embedding models:")
for model in models:
    print(f"- {model}")

Testing 4 embedding models:
- amazon.titan-embed-text-v1
- amazon.titan-embed-text-v2:0
- cohere.embed-english-v3
- cohere.embed-multilingual-v3


In [7]:
def get_embedding(text, model_id):
    """Get embedding for text using specified model"""
    if "titan" in model_id:
        body = json.dumps({"inputText": text})
    elif "cohere" in model_id:
        body = json.dumps({"texts": [text], "input_type": "search_document"})
    
    response = bedrock.invoke_model(
        modelId=model_id,
        body=body,
        contentType="application/json"
    )
    
    result = json.loads(response['body'].read())
    
    if "titan" in model_id:
        return np.array(result['embedding'])
    elif "cohere" in model_id:
        return np.array(result['embeddings'][0])

In [15]:
# Compare embeddings across models
results = []

for model in models:
    print(f"\nTesting {model}...")
    
    try:
        # Get embeddings for each context
        embeddings = {}
        for context_name, text in contexts.items():
            embeddings[context_name] = get_embedding(text, model)
        
        # Compare cooling contexts vs enthusiast contexts
        simple_context = ["simple"]
        cooling_contexts = ["cooling_device", "ceiling_fan"]
        enthusiast_contexts = ["sports_enthusiast", "music_fan"]

        simple_cooling_sim = cosine_similarity(
            [embeddings[simple_context[0]]],
            [embeddings[cooling_contexts[1]]]
        )

        simple_enthusiast_sim = cosine_similarity(
            [embeddings[simple_context[0]]],
            [embeddings[enthusiast_contexts[1]]]
        )
        
        # Calculate similarities within groups
        cooling_sim = cosine_similarity(
            [embeddings[cooling_contexts[0]]], 
            [embeddings[cooling_contexts[1]]]
        )[0][0]
        
        enthusiast_sim = cosine_similarity(
            [embeddings[enthusiast_contexts[0]]], 
            [embeddings[enthusiast_contexts[1]]]
        )[0][0]
        
        # Calculate cross-group similarity
        cross_sim = cosine_similarity(
            [embeddings["cooling_device"]], 
            [embeddings["sports_enthusiast"]]
        )[0][0]
        
        # Determine interpretation strength
        if cooling_sim > enthusiast_sim:
            interpretation = "Better at cooling devices"
        elif enthusiast_sim > cooling_sim:
            interpretation = "Better at enthusiast contexts"
        else:
            interpretation = "Similar for both contexts"

        if simple_cooling_sim > simple_enthusiast_sim:
            bias = "Bias towards cooling devices"
        elif simple_enthusiast_sim > simple_cooling_sim:
            bias = "Bias towards enthusiasts"
        else:
            bias = "No bias"
        
        results.append({
            'Model': model,
            'Cooling Similarity': cooling_sim,
            'Enthusiast Similarity': enthusiast_sim,
            'Cross-group Similarity': cross_sim,
            'Interpretation': interpretation,
            "Bias": bias,
            "Cooling Bias Similarity": simple_cooling_sim,
            "Enthusiast Bias Similarity": simple_enthusiast_sim
        })
        
        print(f"✓ Completed")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        results.append({
            'Model': model,
            'Cooling Similarity': None,
            'Enthusiast Similarity': None,
            'Cross-group Similarity': None,
            'Interpretation': f"Error: {e}"
        })


Testing amazon.titan-embed-text-v1...
✓ Completed

Testing amazon.titan-embed-text-v2:0...
✓ Completed

Testing cohere.embed-english-v3...
✓ Completed

Testing cohere.embed-multilingual-v3...
✓ Completed


In [16]:
# Display results
df = pd.DataFrame(results)
print("\n=== EMBEDDING COMPARISON RESULTS ===")
print(df.to_string(index=False))


=== EMBEDDING COMPARISON RESULTS ===
                       Model  Cooling Similarity  Enthusiast Similarity  Cross-group Similarity            Interpretation                         Bias Cooling Bias Similarity Enthusiast Bias Similarity
  amazon.titan-embed-text-v1            0.618767               0.387634                0.135861 Better at cooling devices Bias towards cooling devices [[0.23419205716298722]]     [[0.1981032067679964]]
amazon.titan-embed-text-v2:0            0.496497               0.188247                0.115726 Better at cooling devices Bias towards cooling devices [[0.23061417610754134]]    [[0.11772351782584703]]
     cohere.embed-english-v3            0.577150               0.503178                0.331171 Better at cooling devices Bias towards cooling devices  [[0.3791629497797667]]    [[0.35786854237469456]]
cohere.embed-multilingual-v3            0.708959               0.525924                0.408887 Better at cooling devices     Bias towards enthusiasts [[0

In [14]:
# Summary insights
print("\n=== KEY INSIGHTS ===")
valid_results = df[df['Cooling Similarity'].notna()]

if len(valid_results) > 0:
    best_disambiguation = valid_results.loc[valid_results['Cross-group Similarity'].idxmin()]
    print(f"Best disambiguation: {best_disambiguation['Model']}")
    print(f"Cross-group similarity: {best_disambiguation['Cross-group Similarity']:.4f}")
    
    print("\nModel strengths:")
    for _, row in valid_results.iterrows():
        print(f"- {row['Model']}: {row['Interpretation']}")
else:
    print("No successful model comparisons completed")


=== KEY INSIGHTS ===
Best disambiguation: amazon.titan-embed-text-v2:0
Cross-group similarity: 0.1157

Model strengths:
- amazon.titan-embed-text-v1: Better at cooling devices
- amazon.titan-embed-text-v2:0: Better at cooling devices
- cohere.embed-english-v3: Better at cooling devices
- cohere.embed-multilingual-v3: Better at cooling devices
